In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习：MLOps阶段6：开始使用Vertex AI批量预测自定义图像模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_custom_image_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_custom_image_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_custom_image_model_batch.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>        
</table>
<br/><br/><br/>

##概述

本教程演示了如何从自定义图像模型进行批量预测。

### 目标

在本教程中，您将学习如何使用自定义图像模型来使用`Vertex AI Batch Prediction`。

本教程使用以下 Google Cloud ML 服务和资源：

- `Vertex AI Batch Prediction`
- `Vertex AI 模型`

执行的步骤包括：

- 从 TensorFlow Hub 下载一个预训练的图像分类模型。
- 将 TensorFlow Hub 模型上传为`Vertex AI 模型`资源。
- 使用原始（未压缩）图像数据进行批量预测到`模型`资源中，使用 JSONL 格式。
- 创建一个服务功能来接收压缩的图像数据，并输出解压缩的预处理数据作为模型输入。
- 将 TensorFlow Hub 模型和服务功能上传为`Vertex AI 模型`资源。
- 使用压缩图像数据对`模型`资源进行批量预测，使用文件列表格式。

在使用批处理预测和在线预测之间有一个关键区别：

* 预测服务：对整个实例集合（即一个或多个数据项）进行按需预测，并即时返回结果。

* 批量预测服务：对整个实例集合进行队列（批量）预测，在后台存储结果，并在准备好时将结果存储在 Cloud Storage 存储桶中。

模型

本教程使用了来自TensorFlow Hub的预训练图像分类模型，该模型经过ImageNet数据集训练。

了解更多关于[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预计使用量生成成本估算。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q
! pip3 install tensorflow $USER_FLAG -q

重新启动内核

安装附加包后，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

*确保在有 GPU 运行时的情况下运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您可以得到 $300 的免费信用额用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、计算引擎 API 和云存储 API。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 会将以 `!` 为前缀的行作为 shell 命令运行，并且会对以 `$` 为前缀的 Python 变量进行插值。

将您的项目 ID 设置为 `gcloud`。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改`REGION`变量，该变量用于笔记本其余部分的操作。以下是Vertex AI支持的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区： `asia-east1`

您不能使用多区域存储桶进行 Vertex AI 的训练。并非所有地区都支持所有 Vertex AI 服务。

了解更多关于 [Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在参加直播教程会话，您可能正在使用共享的测试帐户或项目。为了避免在创建的资源中用户之间的名称冲突，您可以为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源的名称中。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经进行了身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入名称，然后点击**创建**。

在**授予此服务账户对项目访问权限**部分，点击角色下拉列表。在筛选框中键入“Vertex”，选择**Vertex管理员**。在筛选框中键入“Storage Object Admin”，选择**Storage Object Admin**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，输入您的服务帐户密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您使用什么笔记本环境，都必须执行以下步骤。**

当您为Python初始化Vertex AI SDK时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源的关联数据在会话间保留的地方。

在下方设置您的云存储桶的名称。桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容来验证对您的云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

In [ ]:
import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像，以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个带有4个Nvidia Telsa K80 GPU的容器映像分配给每个VM，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

详细了解[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：TF版本在2.3之前的GPU支持将无法加载此教程中的自定义模型。这是一个已知问题，并在TF 2.3中修复。这是由于在服务函数中生成的静态图操作造成的。如果您在自定义模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

设置用于预测的预先构建的Docker容器镜像。

有关最新列表，请参见[用于预测的预先构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.2-5"
else:
    DEPLOY_VERSION = "tf2-cpu.2-5"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的VM的计算资源。
- `机器类型`
     - `n1-standard`：每个虚拟CPU 3.75GB内存。
     - `n1-highmem`：每个虚拟CPU 6.5GB内存。
     - `n1-highcpu`：每个虚拟CPU 0.9GB内存。
 - `虚拟CPU数`：\[2, 4, 8, 16, 32, 64, 96\]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

从TensorFlow Hub获取预训练模型

为了演示目的，本教程使用了从TensorFlow Hub（TFHub）获取的预训练模型，然后将其上传到“Vertex AI 模型”资源中。一旦你拥有了“Vertex AI 模型”资源，该模型就可以部署到“Vertex AI 端点”资源上。

### 下载预训练模型

首先，从TensorFlow Hub下载预训练模型。该模型以TF.Keras层的形式下载。在这个示例中，为了完成模型，您创建了一个带有下载的TFHub模型作为层的“Sequential()”模型，并指定了模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 32, 32, 3])

tfhub_model.summary()

### 保存模型工件

在这一点上，模型已经保存在内存中。接下来，您需要将模型工件保存到云存储位置。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

### 将 TensorFlow Hub 模型上传到「Vertex AI 模型」资源

最后，您可以使用 `upload()` 方法将 TFHub 模型的模型工件上传到「Vertex AI 模型」资源，需要提供以下参数：

- `display_name`：`Model` 资源的人类可读名称。
- `artifact_uri`：模型包的 Cloud Storage 位置。
- `serving_container_image_uri`：服务容器镜像。

将模型上传到 Vertex AI 模型资源会返回一个长时间运行的操作，可能需要一些时间。 

*注意：* 当您将模型工件上传到「Vertex AI 模型」资源时，需要指定相应的部署容器镜像。

In [ ]:
model = aip.Model.upload(
    display_name="resnet_" + UUID,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model)

## 批量预测介绍

批量预测提供了离线批处理大量预测请求的能力。资源仅在批处理过程中进行分配，然后在批量请求完成时取消分配。结果存储在云存储中，与在线预测不同，后者将结果作为HTTP响应包返回。

批处理作业的输入格式取决于您的模型服务器支持的格式。首先，您的模型服务器中的Web服务器必须支持JSONL格式，Web服务器将其转换为模型输入接口或服务函数接口直接支持的格式。对于批量预测，这种JSONL格式被称为“pivot”格式。

### 批量预测作业的输入格式

批处理服务器接受以下输入格式用于自定义图像模型：

- JSONL
- 文件列表

### 批量预测作业的输出格式

批处理服务器接受以下输出格式用于自定义图像模型：

- JSONL

### Pivot格式

批处理服务器将输入格式转换为“pivot”（JSONL）格式，如下所示：

**JSONL**

每个输入行（请求）应包含一个且仅一个有效的JSON值。

    {"values": [1, 2, 3, 4], "key": 1}
    {"values": [5, 6, 7, 8], "key": 2}

批处理服务器以相同格式生成pivot数据。生成的pivot数据然后被包裹成一个负载请求：

    {"instances": [
      {"values": [1, 2, 3, 4], "key": 1},
      {"values": [5, 6, 7, 8], "key": 2}
    ]}

**CSV**

第一行中的csv标题总是被忽略。字符串字段需要被明确用双引号括起来，否则该行将被丢弃，并且解析错误消息将输出到错误文件。非引号值总是作为浮点数传输。

    col1,col2,col3
    1,3,"cat1"
    2,4,"cat2"

批处理服务器将每个输入行（请求）转换为一个JSON数组。

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}
    
**BigQuery**

每行转换为一个JSON数组。例如：

    [1.0,3.0,"cat1"]
    [2.0,4.0,"cat2"]
    
批处理服务器以相同格式生成pivot数据。生成的pivot数据然后被包裹成一个负载请求：

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}

**TFRecords**

TFRecord文件中的实例由apache_beam.io.tfrecordio模块以二进制形式读取。然后将二进制对象序列化为ASCII字符串。预测服务器负责知道解码器来恢复实例。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

**文件列表**

文件列表格式包含文件列表。在“文件列表”文件中，每行指定单个文件路径，指定为云存储位置。

    gs://my-bucket/file1.txt
    gs://my-bucket/file2.txt

批处理服务器将文件作为二进制读取。二进制对象被序列化为ASCII字符串。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

发出一批预测请求
向您的“Vertex AI模型”资源发送一批预测请求。

### 获取测试数据

从CIFAR数据集下载图像并进行预处理。

#### 下载测试图像

从CIFAR数据集下载提供的图像集合。

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

预处理图像

在将测试数据传递给`Vertex AI Batch Prediction`之前，您需要将其进行预处理，以符合模型服务器的预期格式。

`x_test`：
通过将每个像素除以255来规范化（重新缩放）像素数据。这将每个单字节整数像素替换为介于0和1之间的32位浮点数。

In [ ]:
import numpy as np

x_test = (x_test / 255.0).astype(np.float32)

准备批量预测的数据

在你能够运行数据进行批量预测之前，你需要将数据保存为几种可能的格式之一。

在本教程中，使用JSONL格式，因为它与每个图像当前表示的三维列表兼容。为此：

1. 在一个文件中，将每个实例写成一个JSON行。
2. 将此文件上传到云存储。

有关批量预测输入格式的更多详细信息: https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions#batch_request_input

In [ ]:
import json

BATCH_PREDICTION_INSTANCES_FILE = "batch_prediction_instances.jsonl"

BATCH_PREDICTION_GCS_SOURCE = (
    BUCKET_URI + "/batch_prediction_instances/" + BATCH_PREDICTION_INSTANCES_FILE
)

# Write instances at JSONL
with open(BATCH_PREDICTION_INSTANCES_FILE, "w") as f:
    for x in x_test[:10]:
        f.write(json.dumps(x.tolist()) + "\n")

# Upload to Cloud Storage bucket
! gsutil cp $BATCH_PREDICTION_INSTANCES_FILE $BATCH_PREDICTION_GCS_SOURCE

print("Uploaded instances to: ", BATCH_PREDICTION_GCS_SOURCE)

### 发送预测请求

要发送批量预测请求，请使用以下参数调用模型对象的`batch_predict`方法：
- `instances_format`：批量预测请求文件的格式："jsonl"、"csv"、"bigquery"、"tf-record"、"tf-record-gzip"或"file-list"
- `prediction_format`：批量预测响应文件的格式："jsonl"、"csv"、"bigquery"、"tf-record"、"tf-record-gzip"或"file-list"
- `job_display_name`：预测作业的可读名称。
- `gcs_source`：一个或多个云存储路径列表，用于批量预测请求。
- `gcs_destination_prefix`：服务将写入预测结果的云存储路径。
- `model_parameters`：用于提供预测结果的额外过滤参数。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要连接到工作节点的加速器数量。
- `starting_replica_count`：最初预留的计算实例数量。
- `max_replica_count`：扩展到的最大计算实例数量。在本教程中，仅预留一个实例。

### 计算实例的扩展

您可以指定一个单个实例（或节点）来处理您的批量预测请求。本教程使用单个节点，因此变量`MIN_NODES`和`MAX_NODES`均设置为`1`。

如果要使用多个节点来处理批量预测请求，请将`MAX_NODES`设置为您要使用的节点的最大数量。 Vertex AI 会自动调整用于提供预测的节点数量，最多可达到您设置的最大数量。请参考[定价页面](https://cloud.google.com/vertex-ai/pricing#prediction-prices)了解使用多个节点进行自动扩展的成本。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

# The name of the job
BATCH_PREDICTION_JOB_NAME = "cifar10_batch-" + UUID

# Folder in the bucket to write results to
DESTINATION_FOLDER = "batch_prediction_results"

# The Cloud Storage bucket to upload results to
BATCH_PREDICTION_GCS_DEST_PREFIX = BUCKET_URI + "/" + DESTINATION_FOLDER

# Make SDK batch_predict method call
batch_prediction_job = model.batch_predict(
    instances_format="jsonl",
    predictions_format="jsonl",
    job_display_name=BATCH_PREDICTION_JOB_NAME,
    gcs_source=BATCH_PREDICTION_GCS_SOURCE,
    gcs_destination_prefix=BATCH_PREDICTION_GCS_DEST_PREFIX,
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=True,
)

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的云存储输出桶中。您可以调用方法iter_outputs()来获取生成结果的每个云存储文件的列表。每个文件包含一个或多个以JSON格式呈现的预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

bp_iter_outputs = batch_prediction_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

删除批量预测作业

您可以使用 `delete()` 方法删除您的 `Vertex AI Batch Prediction` 作业。

In [ ]:
batch_prediction_job.delete()

#### 删除模型

您可以使用`delete()`方法删除您的`Vertex AI模型`资源。

In [ ]:
model.delete()

## 具有serving功能的图像模型

以前，您的模型服务器将输入作为一个三维数组。图像模型通常接收压缩图像，并使用与模型融合的serving功能将压缩图像解压缩为一个三维数组，并进行其他预处理 -- 比如规范化像素值。

接下来，您将自定义的图像模型上传为一个`Vertex AI Model`资源，并附带一个serving功能。在上传过程中，您需要定义一个serving功能来将数据转换为模型预期的格式。如果您发送编码数据到Vertex AI，您的serving功能会确保该数据在传递到模型输入之前在模型服务器上解码。

### serving功能如何工作

当您向在线预测服务器发送请求时，请求首先会被一个HTTP服务器接收。HTTP服务器从HTTP请求内容中提取出预测请求。而后，提取出的预测请求会被转发到serving功能。对于谷歌预构建的预测容器，请求内容会作为`tf.string`传递给serving功能。

serving功能由两部分组成：

- `预处理功能`：
  - 将输入（`tf.string`）转换为底层模型的输入形状和数据类型（动态图）。
  - 执行与训练底层模型期间相同的数据预处理 -- 如规范化、缩放等。
- `后处理功能`：
  - 将模型输出转换为接收应用程序期望的格式 -- 例如压缩输出。
  - 为接收应用程序打包输出 -- 例如添加标题，组成JSON对象等。

预处理和后处理功能都会转换为与模型融合的静态图。底层模型的输出会传递给后处理功能。后处理功能会将转换/打包后的输出传递回HTTP服务器。HTTP服务器将该输出作为HTTP响应内容返回。

构建用于TF.Keras模型的serving功能时需要考虑一个重要问题，即它们作为静态图运行。这意味着，您不能使用需要动态图的TF图操作。如果使用了必须要动态图的操作，编译serving功能时会出错，提示您正在使用不受支持的EagerTensor。

### 图像数据的服务功能

#### 预处理
为了将图像传递给预测服务，您需要将压缩（例如JPEG）图像字节编码为base 64 - 这样在通过网络传输二进制数据时可以使内容免受修改。由于部署模型期望输入数据为原始（未压缩）字节，您需要确保将base 64编码的数据转换回原始字节，然后进行预处理以匹配模型的输入要求，然后再将其作为输入传递给部署模型。

为了解决这个问题，您需要定义一个服务函数（`serving_fn`）并将其附加到模型上作为预处理步骤。添加`@tf.function`装饰器，使服务函数融合到基础模型中（而不是在CPU上游进行）。

当您发送预测或解释请求时，请求的内容会被base 64解码为一个Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数将`tf.string`预处理为原始（未压缩）的numpy字节（`preprocess_fn`）以匹配模型的输入要求：

- `io.decode_jpeg`- 解压缩JPG图像，返回一个带有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为float 32，并将像素数据重新缩放到0到1之间。
- `image.resize` - 将图像调整大小以匹配模型的输入形状。

此时，数据可以通过一个具体函数传递给模型（`m_call`）。服务函数是一个静态图，而模型是一个动态图。具体函数执行将输入数据从服务函数传递到模型，并将模型的预测结果从模型传递回服务函数的任务。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(tfhub_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(tfhub_model, MODEL_DIR, signatures={"serving_default": serving_fn})

获取serving函数的签名

您可以通过将模型重新加载到内存中，并查询与每个层对应的签名，来获取模型的输入和输出层的签名。

对于您的目的，您需要serving函数的签名。为什么呢？嗯，当我们将数据作为HTTP请求数据包发送进行预测时，图像数据是base64编码的，而我们的TF.Keras模型需要numpy输入。您的serving函数将从base64转换为numpy数组。

在进行预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要知道serving函数的输入层名称 — 之后在进行预测请求时会用到。

In [ ]:
loaded = tf.saved_model.load(MODEL_DIR)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传TensorFlow Hub模型到`Vertex AI模型`资源

最后，您可以使用方法`upload()`将TFHub模型和serving函数的模型工件上传到一个`Vertex AI模型`资源，具有以下参数：

- `display_name`：`Model`资源的人类可读名称。
- `artifact_uri`：模型包的Cloud Storage位置。
- `serving_container_image_uri`：用于serving的容器镜像。

将模型上传到Vertex AI模型资源会返回一个长时间运行的操作，因为可能需要一些时间。

*注意：*当您将模型工件上传到`Vertex AI模型`资源时，请指定相应的部署容器镜像。

In [ ]:
model = aip.Model.upload(
    display_name="resnet_" + UUID,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model)

### 发送批量预测请求

之前，您使用JSONL格式化了批量预测实例，将它们表示为三维数组。这次，您将批量预测实例格式为文件列表，其中文件中的每一行都是一个压缩图像的云存储位置。

为批量预测准备数据

接下来，您将相同的批量预测请求实例格式化为文件列表格式。

In [ ]:
import tensorflow.python.ops.numpy_ops.np_config as np_config

np_config.enable_numpy_behavior()

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

n = 1
for image in x_test[:10]:
    c_image = tf.io.encode_jpeg(image)
    with tf.io.gfile.GFile(BUCKET_URI + f"/images/image{n}.jpg", "wb") as f:
        f.write(c_image.numpy())
    n += 1

BATCH_PREDICTION_INSTANCES_FILE = "batch_prediction_instances.txt"

BATCH_PREDICTION_GCS_SOURCE = (
    BUCKET_URI + "/batch_prediction_instances/" + BATCH_PREDICTION_INSTANCES_FILE
)

with tf.io.gfile.GFile(BATCH_PREDICTION_GCS_SOURCE, "w") as f:
    for n in range(1, 11):
        f.write(BUCKET_URI + f"/images/image{n}.jpg\n")

### 发送预测请求

为了发起批量预测请求，使用以下参数调用模型对象的`batch_predict`方法：
- `instances_format`：批量预测请求文件的格式："jsonl"，"csv"，"bigquery"，"tf-record"，"tf-record-gzip"或"file-list"
- `prediction_format`：批量预测响应文件的格式："jsonl"，"csv"，"bigquery"，"tf-record"，"tf-record-gzip"或"file-list"
- `job_display_name`：预测任务的人类可读名称。
- `gcs_source`：您批量预测请求的一个或多个云存储路径的列表。
- `gcs_destination_prefix`：服务将把预测结果写入的云存储路径。
- `model_parameters`：用于过滤服务预测结果的附加参数。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作进程副本的加速器数量。
- `starting_replica_count`：最初要提供的计算实例数。
- `max_replica_count`：要扩展到的最大计算实例数。在本教程中，只使用一个实例。

### 计算实例扩展

您可以指定一个单独的实例（或节点）来处理您的批量预测请求。本教程使用单个节点，因此变量`MIN_NODES`和`MAX_NODES`都设置为`1`。

如果您想要使用多个节点来处理批量预测请求，请将`MAX_NODES`设置为您要使用的最大节点数。Vertex AI会自动调整用于提供预测的节点数量，最多可以达到您设置的最大数量。请参考[定价页面](https://cloud.google.com/vertex-ai/pricing#prediction-prices)了解使用多个节点进行自动扩展的成本。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

# The name of the job
BATCH_PREDICTION_JOB_NAME = "cifar10_batch-" + UUID

# Folder in the bucket to write results to
DESTINATION_FOLDER = "batch_prediction_results2"

# The Cloud Storage bucket to upload results to
BATCH_PREDICTION_GCS_DEST_PREFIX = BUCKET_URI + "/" + DESTINATION_FOLDER

# Make SDK batch_predict method call
batch_prediction_job = model.batch_predict(
    instances_format="file-list",
    predictions_format="jsonl",
    job_display_name=BATCH_PREDICTION_JOB_NAME,
    gcs_source=BATCH_PREDICTION_GCS_SOURCE,
    gcs_destination_prefix=BATCH_PREDICTION_GCS_DEST_PREFIX,
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=True,
)

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的云存储输出桶中。您可以调用方法 iter_outputs() 来获取生成结果的每个云存储文件的列表。每个文件以 JSON 格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

bp_iter_outputs = batch_prediction_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

为了清理此项目中使用的所有 Google Cloud 资源，您可以[删除用于本教程的Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False
delete_model = True
delete_batch_job = True

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)
if delete_batch_job:
    batch_prediction_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}